In [1]:
import pandas as pd
from mtranslate import translate
from tqdm.notebook import tqdm

In [3]:
df=pd.read_csv("./dataSet/Sentiment_Analysis.csv")

In [3]:
df.shape

(40000, 4)

In [4]:
splitted=df.content[1].split()
filtered=' '.join([word for word in splitted if not word.startswith('@')])
print(filtered)

  

Layin n bed with a headache ughhhh...waitin on your call...


In [16]:
df['tweets']=df.content.apply(lambda x:' '.join([word for word in x.split() if not word.startswith('@')]))

In [19]:
df.to_csv("./dataSet/tweets.csv",index=False)

### getting unrepeated sentiments

In [26]:
sent={}
for sentiment in df.sentiment.unique():
    sent[sentiment]=translate(sentiment,'ckb')

In [27]:
sent

{'empty': 'بەتاڵ',
 'sadness': 'دڵتەنگی',
 'enthusiasm': 'پەرۆشی',
 'neutral': 'بێلایەن',
 'worry': 'نیگەرانی',
 'surprise': 'سوپرایس',
 'love': 'خۆشەویستی',
 'fun': 'خۆش',
 'hate': 'ڕق',
 'happiness': 'بەختەوەری',
 'boredom': 'بێزاری',
 'relief': 'حەسانەوە',
 'anger': 'تووڕەیی'}

In [39]:
sentiments={'empty': 'بەتاڵ',
 'sadness': 'دڵتەنگ',
 'enthusiasm': 'پەرۆش',
 'neutral': 'ئاسایی',
 'worry': 'نیگەران',
 'surprise': 'سوپرایز',
 'love': 'خۆشەویستی',
 'fun': 'خۆشی',
 'hate': 'ڕق',
 'happiness': 'بەختەوەر',
 'boredom': 'بێزار',
 'relief': 'ئارام',
 'anger': 'تووڕە'}

In [40]:
df['kurdish_sentiments']=df.sentiment.apply(lambda x:sentiments[x])
sentiments["empty"]

'بەتاڵ'

In [41]:
df['kurdish_sentiments']

0            بەتاڵ
1           دڵتەنگ
2           دڵتەنگ
3            پەرۆش
4           ئاسایی
           ...    
39995       ئاسایی
39996    خۆشەویستی
39997    خۆشەویستی
39998     بەختەوەر
39999    خۆشەویستی
Name: kurdish_sentiments, Length: 40000, dtype: object

In [44]:
df.head()

,tweet_id,sentiment,author,content,tweets,kurdish_sentiments
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...,i know i was listenin to bad habit earlier and...,بەتاڵ
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin on...,دڵتەنگ
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...,دڵتەنگ
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!,پەرۆش
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...,We want to trade with someone who has Houston ...,ئاسایی


In [2]:
df.to_csv("./dataSet/Tweets_Sentiment.csv",index=False)

NameError: name 'df' is not defined

### Translating tweets

In [1]:
from mtranslate import translate 
import pandas as pd
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
partial_save_amount = 1000

def translate_text(text, to_language='ckb'):
    try:
        return translate(text, to_language)
    except Exception as e:
        print(f'Error translating text: {text} with error {e}')
        return None

def dataset_translator(df):
    res = {}
    futures = {}
    with ThreadPoolExecutor(max_workers=10) as executor: 
        for i, row in df.iterrows():
            future = executor.submit(translate_text, row['tweets'], 'ckb')
            futures[future] = row['tweet_id']  # Map future to tweet_id

        for future in tqdm(as_completed(futures), total=len(futures)):
            result = future.result()
            if result is not None:
                myid = futures[future]  # Use the mapping to get the correct tweet_id
                res[myid] = result

            # Check if it's time to save partial results
            if len(res) % partial_save_amount == 0:
                results = pd.DataFrame.from_dict(res, orient='index')
                results = results.reset_index().rename(columns={'index': 'tweet_id', 0: 'kurdish_tweets'})
                results = results.merge(df.iloc[:len(res)], on='tweet_id', how='right')
                results.to_csv(f"./dataSet/translated/kurdish_tweets-{len(res)}.csv", index=False)
                print(f'saved at {len(res)}')
              

    # After all futures are completed, save any remaining translations
    if res:
        results = pd.DataFrame.from_dict(res, orient='index')
        results = results.reset_index().rename(columns={'index': 'tweet_id', 0: 'kurdish_tweets'})
        results = results.merge(df, on='tweet_id', how='right')
        results.to_csv(f"./dataSet/translated/kurdish_tweets_final.csv", index=False)
        print(f'Final save at {len(res)}')

    return pd.DataFrame.from_dict(res, orient='index')

In [2]:
data=pd.read_csv("./dataSet/Tweets_Sentiment.csv")


In [3]:
kurdish_dataset = dataset_translator(data.iloc[38000:])

  0%|          | 0/2000 [00:00<?, ?it/s]

Error translating text: nan with error quote_from_bytes() expected bytes
Error translating text: nan with error quote_from_bytes() expected bytes
saved at 1000
Error translating text: nan with error quote_from_bytes() expected bytes
Error translating text: nan with error quote_from_bytes() expected bytes
Error translating text: nan with error quote_from_bytes() expected bytes
Final save at 1995


In [2]:
translate("I'm pretty sure Isis has a tapeworm",'ckb')

'من زۆر دڵنیام ئیسیس کرمی شریتی هەیە'

,tweet_id,sentiment,author,content,tweets,kurdish_sentiments
3000,1957674180,worry,blue_cat_online,[BBC] Malaria parasites becoming resistant to ...,[BBC] Malaria parasites becoming resistant to ...,نیگەران
3001,1957674579,worry,tickvg,Morning all. I appear to have a bit of a sore ...,Morning all. I appear to have a bit of a sore ...,نیگەران
3002,1957674594,surprise,ashleigh_03,has just booked her appointment to give blood ...,has just booked her appointment to give blood ...,سوپرایز
3003,1957674693,hate,barbsrad,"urgh, my head is banging.","urgh, my head is banging.",ڕق
3004,1957674793,neutral,lilianerausch,@Donnette i am fine. just having a headache fo...,i am fine. just having a headache for days now,ئاسایی
...,...,...,...,...,...,...
39995,1753918954,neutral,showMe_Heaven,@JohnLloydTaylor,NaN,ئاسایی
39996,1753919001,love,drapeaux,Happy Mothers Day All my love,Happy Mothers Day All my love,خۆشەویستی
39997,1753919005,love,JenniRox,Happy Mother's Day to all the mommies out ther...,Happy Mother's Day to all the mommies out ther...,خۆشەویستی
39998,1753919043,happiness,ipdaman1,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...,WASSUP BEAUTIFUL!!! FOLLOW ME!! PEEP OUT MY NE...,بەختەوەر


In [12]:
def csv_concat(df1,df2,save_path="./dataSet/translated/kurdish_tweets-40000-concat.csv"):
    data_frame1=pd.read_csv(df1)
    data_frame2=pd.read_csv(df2)
    combined=pd.concat([data_frame1,data_frame2])
    combined.to_csv(save_path,index=False)


csv_concat("./dataSet/translated/kurdish_tweets-38000-concat.csv","./dataSet/translated/kurdish_tweets-40000-saved.csv")


In [13]:
dc=pd.read_csv("./dataSet/translated/kurdish_tweets-40000-concat.csv")

In [14]:
dc.shape

(40000, 7)